In [1]:
from idaes.dmf import DMF
from idaes.dmf import resource
import pandas

In [2]:
# Import Pyomo libraries
from pyomo.environ import Constraint, Expression, log, NonNegativeReals,\
    value, Var, Set, Param, sqrt, log10
from pyomo.opt import SolverFactory, TerminationCondition
from pyomo.util.calc_var_value import calculate_variable_from_constraint

# Import IDAES cores
from idaes.core import (declare_process_block_class,
                        MaterialFlowBasis,
                        PhysicalParameterBlock,
                        StateBlockData,
                        StateBlock)
from idaes.core.util.initialization import solve_indexed_blocks
from idaes.core.util.misc import add_object_reference
from idaes.core.util.model_statistics import degrees_of_freedom, \
                                             number_unfixed_variables
from idaes.core.util.misc import extract_data

In [3]:
wspace = DMF(path='.')

In [5]:
r = wspace.find_one(name='pressure_crit')
print(r.data)
td = resource.TidyUnitData(**r.data)
print(td.param_data)

{'variables': ['compound', 'pressure'], 'units': [None, 'Pa'], 'observations': [['benzene', 4890000.0], ['toluene', 4100000.0], ['hydrogen', 1290000.0], ['methane', 4600000.0], ['diphenyl', 3850000.0]]}
{'benzene': 4890000.0, 'toluene': 4100000.0, 'hydrogen': 1290000.0, 'methane': 4600000.0, 'diphenyl': 3850000.0}


In [6]:
component_list = Set(initialize=['benzene',
                                              'toluene',
                                              'hydrogen',
                                              'methane'])

In [7]:
temperature_crit = Param(
            component_list,
            within=NonNegativeReals,
            mutable=False,
            initialize=extract_data(td.param_data),
            doc='Critical temperature [K]')

In [8]:
r = resource.Resource(type_ = resource.TY_PROPERTY)

In [48]:
r.data = dict(variables=('compound', 'pressure'), units=(None, 'Pa'), 
                      observations=[['benzene', 48.9e5],['toluene', 41e5],
                              ['hydrogen', 12.9e5],
                              ['methane', 46e5],
                              ['diphenyl', 38.5e5]
                         ])

In [13]:
reid_source = dict(date='1987', isbn='9780070517998', language='en', 
                   source="Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids.")
r.v["sources"].append(reid_source)

In [14]:
r.v["sources"]

[{'date': '1987',
  'isbn': '9780070517998',
  'language': 'en',
  'source': 'Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids.'}]

In [19]:
r.v['tags'] = ['pressure', 'critical pressure']

In [20]:
r.v['aliases'] = ['pressure_crit']

In [52]:
wspace.add(r)

'623804ff69564eda9f92ae3871ed8687'

In [4]:
reid_source = dict(date='1987', isbn='9780070517998', language='en', 
                   source="Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids.")
perry_source = dict(date='2019', isbn='9780071834087', language='en', 
                   source="Green, W. and Southard, M., ed. 'Perry's Chemical Engineer's Handbook', 9th Edition")

In [10]:
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'temperature'), units=(None, 'K'), 
                      observations=[
                          ['benzene', 562.2],
                        ['toluene', 591.8],
                        ['hydrogen', 33.0],
                        ['methane', 190.4],
                        ['diphenyl', 789]
                         ])
r.v["sources"].append(reid_source)
r.v['tags'] = ['temperature', 'critical']
r.v['aliases'] = 'temperature_crit'

In [11]:
r.v

{'id_': 'b0b333fdd4574e4eae1c4cef98c19ee8',
 'type': 'propertydb',
 'aliases': 'temperature_crit',
 'codes': [],
 'collaborators': [],
 'created': 1565634717.614104,
 'modified': 1565634717.614104,
 'creator': {'name': 'dang'},
 'data': {'variables': ('compound', 'temperature'),
  'units': (None, 'K'),
  'observations': [['benzene', 562.2],
   ['toluene', 591.8],
   ['hydrogen', 33.0],
   ['methane', 190.4],
   ['diphenyl', 789]]},
 'datafiles': [],
 'datafiles_dir': '',
 'desc': '',
 'relations': [],
 'sources': [{'date': '1987',
   'isbn': '9780070517998',
   'language': 'en',
   'source': 'Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids.'}],
 'tags': ['temperature', 'critical'],
 'version_info': {'created': 1565634717.614104,
  'version': (0, 0, 0),
  'name': ''}}

In [12]:
wspace.add(r)

'b0b333fdd4574e4eae1c4cef98c19ee8'

In [5]:
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'molecular weight'), units=(None, 'Kg/mol'), 
                      observations=[
                        ['benzene', 78.1136E-3],
                        ['toluene', 92.1405E-3],
                        ['hydrogen', 2.016e-3],
                        ['methane', 16.043e-3],
                        ['diphenyl', 154.212e-4]
                         ])
r.v["sources"].append(reid_source)
r.v['tags'] = ['mw', 'molecular', 'weight']
r.v['aliases'] = 'mw'
r.v['desc'] = 'Molecular weight'

In [6]:
wspace.add(r)

'd9b59dffe0e4428baab2f80f42c62c3c'

In [7]:
wspace.update?

Signature: wspace.update(rsrc, sync_relations=False, upsert=False)
Docstring:
Update/insert stored resource.

Args:
    rsrc (resource.Resource): Resource instance
    sync_relations (bool): If True, and if resource
        exists in the DB, then the "relations" attribute of
        the provided resource will be changed to the stored value.
    upsert (bool): If true, and the resource is not in the DMF,
                   then insert it. If false, and the resource is
                   not in the DMF, then do nothing.
Returns:
    bool: True if the resource was updated or added, False if nothing
          was done.
Raises:
    errors.DMFError: If the input resource was invalid.
File:      ~/src/idaes/dangunter/idaes-dev/idaes/dmf/dmfbase.py
Type:      method


In [8]:
r_pressure = wspace.find_one(name="pressure_crit")

In [9]:
r_pressure

In [11]:
r_pressure.v['desc'] = "Critical pressure"

In [12]:
wspace.update(r_pressure)

False

In [17]:
r_temp = wspace.fetch_one('b0b333fdd4574e4eae1c4cef98c19ee8')

In [18]:
r_temp

In [20]:
r_temp.v['desc'] = "Critical temperature"

In [21]:
wspace.update(r_temp)

False

In [29]:
obs_dict = {('benzene', '1'): 1.0162,
                         ('benzene', '2'): 0.2655,
                         ('benzene', '3'): 562.16,
                         ('benzene', '4'): 0.28212,
                         ('toluene', '1'): 0.8488,
                         ('toluene', '2'): 0.26655,
                         ('toluene', '3'): 591.8,
                         ('toluene', '4'): 0.2878,
                         ('hydrogen', '1'): 5.414,
                         ('hydrogen', '2'): 0.34893,
                         ('hydrogen', '3'): 33.19,
                         ('hydrogen', '4'): 0.2706,
                         ('methane', '1'): 2.9214,
                         ('methane', '2'): 0.28976,
                         ('methane', '3'): 190.56,
                         ('methane', '4'): 0.28881,
                         ('diphenyl', '1'): 0.5039,
                         ('diphenyl', '2'): 0.25273,
                         ('diphenyl', '3'): 789.26,
                         ('diphenyl', '4'): 0.281}
obs = [[k[0], k[1], v] for k, v in obs_dict.items()] 
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'n', 'Liquid density'), units=(None, None, None), 
                      observations=obs)
r.v["sources"].append(reid_source)
r.v['tags'] = ['liquid', 'density']
r.v['aliases'] = 'cp_liq'
r.v['desc'] = 'Liquid densities'

In [32]:
wspace.add(r)

'30ad5a72c576471b8b0a6efcab1114c8'

In [36]:
bp_data = {('benzene'): 353.25,
                   ('toluene'): 383.95,
                   ('hydrogen'): 20.45,
                   ('methane'): 111.75,
                   ('diphenyl'): 528.05}
obs = [[k, v] for k, v in bp_data.items()]
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'temperature'), units=(None, 'K'), 
                      observations=obs)
r.v['sources'].append(perry_source)
r.v['tags'] = ['boiling', 'boiling point']
r.v['aliases'] = 'bp'
r.v['desc'] = 'Pure component boiling points at standard pressure [K]'

In [37]:
r.v

{'id_': '4ef9d2ce193f4a0487763fbe4b94c347',
 'type': 'propertydb',
 'aliases': 'bp',
 'codes': [],
 'collaborators': [],
 'created': 1565647456.847174,
 'modified': 1565647456.847174,
 'creator': {'name': 'dang'},
 'data': {'variables': ('compound', 'temperature'),
  'units': (None, 'K'),
  'observations': [['benzene', 353.25],
   ['toluene', 383.95],
   ['hydrogen', 20.45],
   ['methane', 111.75],
   ['diphenyl', 528.05]]},
 'datafiles': [],
 'datafiles_dir': '',
 'desc': 'Pure component boiling points at standard pressure [K]',
 'relations': [],
 'sources': [{'date': '2019',
   'isbn': '9780071834087',
   'language': 'en',
   'source': "Green, W. and Southard, M., ed. 'Perry's Chemical Engineer's Handbook', 9th Edition"}],
 'tags': ['boiling', 'boiling point'],
 'version_info': {'created': 1565647456.847174,
  'version': (0, 0, 0),
  'name': ''}}

In [38]:
wspace.add(r)

'4ef9d2ce193f4a0487763fbe4b94c347'

In [39]:
cp_ig_data = {('Liq', 'benzene', '1'): 1.29E5,
                      ('Liq', 'benzene', '2'): -1.7E2,
                      ('Liq', 'benzene', '3'): 6.48E-1,
                      ('Liq', 'benzene', '4'): 0,
                      ('Liq', 'benzene', '5'): 0,
                      ('Vap', 'benzene', '1'): -3.392E1,
                      ('Vap', 'benzene', '2'): 4.739E-1,
                      ('Vap', 'benzene', '3'): -3.017E-4,
                      ('Vap', 'benzene', '4'): 7.130E-8,
                      ('Vap', 'benzene', '5'): 0,
                      ('Liq', 'toluene', '1'): 1.40E5,
                      ('Liq', 'toluene', '2'): -1.52E2,
                      ('Liq', 'toluene', '3'): 6.95E-1,
                      ('Liq', 'toluene', '4'): 0,
                      ('Liq', 'toluene', '5'): 0,
                      ('Vap', 'toluene', '1'): -2.435E1,
                      ('Vap', 'toluene', '2'): 5.125E-1,
                      ('Vap', 'toluene', '3'): -2.765E-4,
                      ('Vap', 'toluene', '4'): 4.911E-8,
                      ('Vap', 'toluene', '5'): 0,
                      ('Liq', 'hydrogen', '1'): 0,  # 6.6653e1,
                      ('Liq', 'hydrogen', '2'): 0,  # 6.7659e3,
                      ('Liq', 'hydrogen', '3'): 0,  # -1.2363e2,
                      ('Liq', 'hydrogen', '4'): 0,  # 4.7827e2, # Eqn 2
                      ('Liq', 'hydrogen', '5'): 0,
                      ('Vap', 'hydrogen', '1'): 2.714e1,
                      ('Vap', 'hydrogen', '2'): 9.274e-3,
                      ('Vap', 'hydrogen', '3'): -1.381e-5,
                      ('Vap', 'hydrogen', '4'): 7.645e-9,
                      ('Vap', 'hydrogen', '5'): 0,
                      ('Liq', 'methane', '1'): 0,  # 6.5708e1,
                      ('Liq', 'methane', '2'): 0,  # 3.8883e4,
                      ('Liq', 'methane', '3'): 0,  # -2.5795e2,
                      ('Liq', 'methane', '4'): 0,  # 6.1407e2, # Eqn 2
                      ('Liq', 'methane', '5'): 0,
                      ('Vap', 'methane', '1'): 1.925e1,
                      ('Vap', 'methane', '2'): 5.213e-2,
                      ('Vap', 'methane', '3'): 1.197e-5,
                      ('Vap', 'methane', '4'): -1.132e-8,
                      ('Vap', 'methane', '5'): 0,
                      ('Liq', 'diphenyl', '1'): 1.2177e5,
                      ('Liq', 'diphenyl', '2'): 4.2930e2,
                      ('Liq', 'diphenyl', '3'): 0,
                      ('Liq', 'diphenyl', '4'): 0,
                      ('Liq', 'diphenyl', '5'): 0,
                      ('Vap', 'diphenyl', '1'): -9.707e1,
                      ('Vap', 'diphenyl', '2'): 1.106e0,
                      ('Vap', 'diphenyl', '3'): -8.855e-4,
                      ('Vap', 'diphenyl', '4'): 2.790e-7,
                      ('Vap', 'diphenyl', '5'): 0}

In [40]:
obs = [[k[0], k[1], k[2], v] for k, v in cp_ig_data.items()]
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('phase', 'compound', 'n', 'value'), units=(None, None, None, None), 
                      observations=obs)
r.v['sources'].append(perry_source)
r.v['tags'] = ['Cp', 'heat capacity', 'enthalpy']
r.v['aliases'] = 'cp'
r.v['desc'] = 'Specific heat capacity, enthalpy'

In [42]:
wspace.add(r)

'5bc493a1ccb74639b0403d0d66924b51'

In [43]:
pressure_sat_coeff_data = {('benzene', 'A'): 4.202,
                                   ('benzene', 'B'): 1322,
                                   ('benzene', 'C'): -38.56,
                                   ('toluene', 'A'): 4.216,
                                   ('toluene', 'B'): 1435,
                                   ('toluene', 'C'): -43.33,
                                   ('hydrogen', 'A'): 3.543,
                                   ('hydrogen', 'B'): 99.40,
                                   ('hydrogen', 'C'): 7.726,
                                   ('methane', 'A'): 3.990,
                                   ('methane', 'B'): 443.0,
                                   ('methane', 'C'): -0.49,
                                   ('diphenyl', 'A'): 4.345,
                                   ('diphenyl', 'B'): 1988,
                                   ('diphenyl', 'C'): -70.82}

In [44]:
obs = [[k[0], k[1], v] for k, v in pressure_sat_coeff_data.items()]
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'letter', 'value'), units=(None, None, None, None), 
                      observations=obs)
r.v['sources'].append(reid_source)
r.v['tags'] = ['pressure', 'saturation']
r.v['aliases'] = 'pressure_sat_coeff'
r.v['desc'] = 'Pressure saturation coefficient'

In [45]:
wspace.add(r)

'6ff47009063047c58d70779374175d9c'

In [8]:
dh_vap = {'benzene': 3.387e4, 'toluene': 3.8262e4,
                  'hydrogen': 0, 'methane': 0, "diphenyl": 6.271e4}

In [9]:
obs = [[k, v] for k, v in dh_vap.items()]
r = resource.Resource(type_ = resource.TY_PROPERTY)
r.data = dict(variables=('compound', 'value'), units=(None, 'K'), 
                      observations=obs)
r.v['sources'].append(reid_source)
r.v['tags'] = ['heat', 'vaporization']
r.v['aliases'] = ['dh_vap']
r.v['desc'] = 'Heat of vaporization'

In [10]:
wspace.add(r)

'15907525b89643c08bf9bc0a0b8e2fe6'

In [5]:
r